### PROJECT SUMMARY:

Goal of this project is to conduct an RFM (recency, frequency, monetary) analysis to segment customers based on purchase behaviour
- Recency - How long ago did the customer last make a purchase (# of days)?
- Frequency - How many purchases did the customer make in the last 90 days?
- Monetary - How much did the customer spend in the last 90 days?


Building off prior exploratory analysis done in SQL

### Step 1: Load and prepare data

In [ ]:
import pandas as pd

df = pd.read_csv('rfm_export.csv')

print(df.head()) # checking import
print(df.dtypes) # checking data types- need to update recency to datetime

   customer_id most_recent_purchase  orders  revenue
0           87           2023-12-21      10   8805.0
1          116           2023-10-11       2    320.0
2           71           2023-12-22       6   2425.0
3           68           2023-12-05       4    380.0
4           51           2023-12-15       7   1650.0
customer_id               int64
most_recent_purchase     object
orders                    int64
revenue                 float64
dtype: object


In [12]:
#convert date column to datetime
df['most_recent_purchase'] = pd.to_datetime(df['most_recent_purchase'])
print(df.dtypes)

customer_id                      int64
most_recent_purchase    datetime64[ns]
orders                           int64
revenue                        float64
dtype: object


In [13]:
# calculating recency of each customer 
    # need to find days since last purchase (currently R is the actual date of last purchase)

# setting reference date to last date in the dataset + 1 day
reference_date = df['most_recent_purchase'].max() + pd.Timedelta(days=1)

# calculating difference
df['recency'] = (reference_date - df['most_recent_purchase']).dt.days

print(df.head())

   customer_id most_recent_purchase  orders  revenue  recency
0           87           2023-12-21      10   8805.0       11
1          116           2023-10-11       2    320.0       82
2           71           2023-12-22       6   2425.0       10
3           68           2023-12-05       4    380.0       27
4           51           2023-12-15       7   1650.0       17


### Step 2: Build RFM table

In [14]:
# creating RFM dataframe
rfm_df = df[['customer_id','recency', 'orders', 'revenue']].copy() # copying customer_id, recency, orders, revenue into new df
rfm_df.rename(columns={'orders':'frequency', 'revenue':'monetary'}, inplace=True) # renaming columns for RFM

print(rfm_df.head())

   customer_id  recency  frequency  monetary
0           87       11         10    8805.0
1          116       82          2     320.0
2           71       10          6    2425.0
3           68       27          4     380.0
4           51       17          7    1650.0


### Step 3: Score customers (R, F, M)

In [15]:
# creating bins to score each datapoint out of 5
rfm_df['rscore'] = pd.cut(rfm_df['recency'],
                           bins=[0,30,60,90,120,float('inf')],
                           labels=[5,4,3,2,1]) # if last purchase within 30 days score 5, 60 days score 4, etc.
rfm_df['fscore'] = pd.qcut(rfm_df['frequency'], q=5, labels=[1,2,3,4,5]) # splits freq into 5 even intervals [CHECK]
rfm_df['mscore'] = pd.qcut(rfm_df['monetary'], q=5, labels=[1,2,3,4,5]) # splits monetary into 5 even intervals [CHECK]


In [16]:
# new df with just scores
rfm_scores = rfm_df[['customer_id','rscore','fscore','mscore']].copy()

print(rfm_scores.dtypes) # checking dtypes

# convert dtypes
rfm_scores[['rscore', 'fscore', 'mscore']] = (
    rfm_scores[['rscore', 'fscore', 'mscore']].apply(pd.to_numeric)
)

print(rfm_scores.dtypes) # confirming change

customer_id       int64
rscore         category
fscore         category
mscore         category
dtype: object
customer_id      int64
rscore           int64
fscore           int64
mscore         float64
dtype: object


### Step 4: Weighted score and segmentation

In [ ]:
# calculating weighted avergage
rfm_scores['weighted_avg'] = (rfm_scores['rscore'] * 0.4 +
                              rfm_scores['fscore'] * 0.3 +
                              rfm_scores['mscore'] * 0.3)
    # rating recency slightly higher, assuming lack of recent activity is a stronger signal on retention risk

print(rfm_scores.sort_values(['weighted_avg'], ascending=False))

     customer_id  rscore  fscore  mscore  weighted_avg
0             87       5       5     5.0           5.0
140           76       5       5     5.0           5.0
36            82       5       5     5.0           5.0
144          110       5       5     5.0           5.0
37           113       5       5     5.0           5.0
..           ...     ...     ...     ...           ...
145          145       1       1     NaN           NaN
147          148       1       1     NaN           NaN
150          129       1       1     NaN           NaN
151          143       1       1     NaN           NaN
152           23       1       1     NaN           NaN

[155 rows x 5 columns]


In [18]:
# cleaning nulls
    # we were expecting nulls as some customers didn't have orders in the date range

print(rfm_scores['weighted_avg'].isna().sum()) # counting how many values are null
rfm_scores = rfm_scores.dropna(subset=['weighted_avg']) # removing null values
print(rfm_scores['weighted_avg'].isna().sum()) # ensuring number of nulls now = 0

21
0


In [ ]:
## segmentation
print(pd.cut(rfm_scores['weighted_avg'], bins = [0,2.5,3.5,4.5,float('inf')], 
             labels = ['poor', 'okay', 'good', 'best'],
             right = False).value_counts(sort=False)) # based on weighted avgs we can see division of customers.

weighted_avg
poor    27
okay    34
good    46
best    27
Name: count, dtype: int64


### Step 5: Takeaways

We find 27 'best'  and '34' good customers based on weighted RFM scores, versus 73 in 'okay' and 'poor'.
This suggests roughly a third of the customer base is already highly valuable

Further segmentation would be helpful for more targetted actions
Potential further segments
- **High M, low R: past big spenders**
    - create marketing campaign to bring them back on. Figure out why they dropped off.
- **High F, low M: frequent but small purchases**
    - how do we make them spend more $$? What products are they frequently purchasing?
- **High R, low F: new customers**
    - how do we turn them into loyal customers? loyalty offers?